In [1]:
from os import environ, path, walk
from pocketsphinx import *
from sphinxbase import *
import fnmatch
import json
from jiwer import wer
import math

relative_path = "../../td_corpus_digits"
pwd = os.getcwd()
previous = len("2SpeakerGroup")
pwd = pwd[:len(pwd)-previous]
#wer -i SNR35dB_Manrulen.ref SNR35dB_Manrulen.hyp > SNR35dB_Manrulen.res

In [2]:
def find_files(filename, search_path):
   result = []

   # Wlaking top-down from the root
   for root, dir, files in os.walk(search_path):
      for file in files:
         if file.__contains__(".raw"):        
            result.append(os.path.join(root, file))
   return result

def calculate_CI(wer, noSen):
    return 1.960 * math.sqrt((wer*(1-wer))/noSen)

def create_decoder_digits(dic, grammar, rule):
    config = Decoder.default_config()
    config.set_string('-hmm', pwd +'ps_data/model/en-us')  # acoustic model
    config.set_string('-dict', pwd +'ps_data/lex/' + dic)  # lexicon / dictionary
    decoder_digit = Decoder(config)

    # Now we use a custom language model
    # Prepare the grammar to be used
    jsgf = Jsgf(pwd +'ps_data/jsgf/' + grammar)  # load the grammar file
    rule = jsgf.get_rule('digits.' + rule)  # choose the rule
    fsg = jsgf.build_fsg(rule, decoder_digit.get_logmath(), 7.5)  # build the grammar rule
    fsg.writefile('../outputs/' + grammar)  # write the compiled grammar rule as an external file

    # Now set the fsg grammar rule in the decoder
    decoder_digit.set_fsg("../outputs/" + grammar, fsg)  # load the pre-recorded compiled grammar rule in the decoder
    decoder_digit.set_search("../outputs/" + grammar)  # and set it as the grammar to use

    return decoder_digit

In [3]:
def rundecoder(file_path, decoder):
    # Start the decoder
    decoder.start_utt()

    # Open the file to decode
    stream = open(file_path, 'rb')
    uttbuf = stream.read(-1)

    # Process the file with the decoder
    if uttbuf:
        decoder.process_raw(uttbuf, False, True)
    else:
        print("Error reading speech data")
        exit()
    decoder.end_utt()

    # test for empty hypothesis and replace the output with an empty string if needed
    if decoder.hyp() is None:
        best_hypothesis = ''
    else:
        best_hypothesis = decoder.hyp().hypstr

    #reading ref file for WER
    with open(file_path[:len(file_path)-3] + "ref") as f:
        actualY = f.readlines()
    
    wordErrorRate = wer(actualY, best_hypothesis)

    return {
    "fileName": file_path.split("/")[-1],
    "prediction": best_hypothesis,
    "actual": actualY[0].replace("\n", ""), 
    # "confidence": decoder.get_logmath().exp(decoder.hyp().prob),
    "wer" : wordErrorRate}

In [4]:
def processWav(waves, rule, group=""):
    decoder = create_decoder_digits("digits.dict", "digits.jsgf", rule)
    ruleN_result = []
    for wav in waves:
        ruleN_result.append(rundecoder(wav, decoder))

    #this is to dump hyp
    with open(group + rule + '.hyp', 'w') as f:
        for i in ruleN_result:
            f.write(i["prediction"])
            f.write("\n")

    #this is to dump ref
    with open(group + rule + '.ref', 'w') as f:
        for i in ruleN_result:
            f.write(i["actual"])
            f.write("\n")

In [ ]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq1digit_200_files/"
man1digit = find_files(".raw",folder_path)
processWav(man1digit, "rule1", group="man_")
folder_path = pwd + "td_corpus_digits/SNR35dB/woman/seq1digit_200_files/"
woman1digit = find_files(".raw",folder_path)
processWav(woman1digit, "rule1", group="woman_")
folder_path = pwd + "td_corpus_digits/SNR35dB/boy/seq1digit_200_files/"
boy1digit = find_files(".raw",folder_path)
processWav(boy1digit, "rule1", group="boy_")
folder_path = pwd + "td_corpus_digits/SNR35dB/girl/seq1digit_200_files/"
girl1digit = find_files(".raw",folder_path)
processWav(girl1digit, "rule1", group="girl_")

In [9]:
conf_interval_man_rule1 = calculate_CI(0.005, 200)
print(conf_interval_man_rule1*100)
conf_interval_woman_rule1 = calculate_CI(0.050, 200)
print(conf_interval_woman_rule1*100)
conf_interval_girl_rule1 = calculate_CI(0.2850, 200)
print(conf_interval_girl_rule1*100)
conf_interval_boy_rule1 = calculate_CI(0.170, 200)
print(conf_interval_boy_rule1*100)

0.9775469298197402
3.0205628614547986
6.256284999902098
5.206004994235022


In [ ]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq3digits_100_files/"
man3digit = find_files(".raw",folder_path)
processWav(man3digit, "rule3", group="man_")
folder_path = pwd + "td_corpus_digits/SNR35dB/woman/seq3digits_100_files/"
woman3digit = find_files(".raw",folder_path)
processWav(woman3digit, "rule3", group="woman_")
folder_path = pwd + "td_corpus_digits/SNR35dB/boy/seq3digits_100_files/"
boy3digit = find_files(".raw",folder_path)
processWav(boy3digit, "rule3", group="boy_")
folder_path = pwd + "td_corpus_digits/SNR35dB/girl/seq3digits_100_files/"
girl3digit = find_files(".raw",folder_path)
processWav(girl3digit, "rule3", group="girl_")

In [10]:
conf_interval_man_rule3 = calculate_CI(0.01667, 100)
print(conf_interval_man_rule3*100)
conf_interval_woman_rule3 = calculate_CI(0.05, 100)
print(conf_interval_woman_rule3*100)
conf_interval_girl_rule3 = calculate_CI(0.18667, 100)
print(conf_interval_girl_rule3*100)
conf_interval_boy_rule3 = calculate_CI(0.150, 100)
print(conf_interval_boy_rule3*100)

2.5094209292536003
4.27172096466986
7.63706929078007
6.998599859971994


In [ ]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq5digits_100_files/"
man5digit = find_files(".raw",folder_path)
processWav(man5digit, "rule5", group="man_")
folder_path = pwd + "td_corpus_digits/SNR35dB/woman/seq5digits_100_files/"
woman5digit = find_files(".raw",folder_path)
processWav(woman5digit, "rule5", group="woman_")
folder_path = pwd + "td_corpus_digits/SNR35dB/boy/seq5digits_100_files/"
boy5digit = find_files(".raw",folder_path)
processWav(boy5digit, "rule5", group="boy_")
folder_path = pwd + "td_corpus_digits/SNR35dB/girl/seq5digits_100_files/"
girl5digit = find_files(".raw",folder_path)
processWav(girl5digit, "rule5", group="girl_")

In [11]:
conf_interval_man_rule5 = calculate_CI(0.01, 100)
print(conf_interval_man_rule5*100)
conf_interval_woman_rule5 = calculate_CI(0.022, 100)
print(conf_interval_woman_rule5*100)
conf_interval_girl_rule5 = calculate_CI(0.2060, 100)
print(conf_interval_girl_rule5*100)
conf_interval_boy_rule5 = calculate_CI(0.1240, 100)
print(conf_interval_boy_rule5*100)

1.950175376728975
2.874993314774836
7.926837089280944
6.459798436483913
